In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_fixed_cost_runs(acq, dim, kernel, lengthscale):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "8ql0ig3e",
        "config.amplitude": 1,
        "config.dim": dim,
        "config.kernel": kernel,
        "config.lengthscale": lengthscale,  
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
    'RandomSearch': 'RandomSearch',
    'ExpectedImprovement':'ExpectedImprovement',
    'ThompsonSampling':'ThompsonSampling', 
    'KnowledgeGradient':'KnowledgeGradient',
    # 'Gittins_Lambda_01':'Gittins_Lambda01',
    # 'Gittins_Lambda_001':'Gittins_Lambda001',
    'Gittins_Lambda_0001':'Gittins_Lambda0001',
    'Gittins_Step_Divide2':'Gittins_Step_Divide2',
    # 'Gittins_Step_Divide5':'Gittins_Step_Divide5',
    # 'Gittins_Step_Divide10':'Gittins_Step_Divide10',
    # 'Gittins_Step_EIpu':'Gittins_Step_EIPC' , 
    }
dimensions = [4, 8, 16, 32]
kernels = ["Matern32", "Matern52", "RBF"]
lengthscales = [1.0, 0.1]

In [4]:
grouped_runs = {(a,d,k,l): load_bayesian_regret_fixed_cost_runs(a,d,k,l) for a in acquisition_functions.keys() for d in (dimensions) for k in (kernels) for l in (lengthscales)}

100%|██████████| 16/16 [00:21<00:00,  1.35s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 16/16 [00:21<00:00,  1.34s/it]


In [4]:
for a in acquisition_functions.keys():
    for d in dimensions:
        for k in kernels:
            for l in lengthscales:
                config_and_metrics_per_seed = grouped_runs[a,d,k,l]
        
                cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                # Handling potential empty arrays
                if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
                    continue  # Skip this iteration if there's no data
                global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0])
        
                regret_per_seed = global_optimum_per_seed - best_observed_per_seed
                print(a, d, k, l, regret_per_seed.shape)
        
                regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
                regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
                regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)
        
                output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)
        
                np.savetxt(f"results/quartiles/fixed_amplitude/BayesianRegret_FixedCost_d{d}_{k}_ls{l}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

RandomSearch 4 Matern32 1.0 (41, 16)
RandomSearch 4 Matern32 0.1 (101, 16)
RandomSearch 4 Matern32 0.01 (101, 16)
RandomSearch 4 Matern52 1.0 (41, 16)
RandomSearch 4 Matern52 0.1 (101, 16)
RandomSearch 4 Matern52 0.01 (101, 16)
RandomSearch 4 RBF 1.0 (21, 16)
RandomSearch 4 RBF 0.1 (81, 16)
RandomSearch 4 RBF 0.01 (81, 16)
RandomSearch 8 Matern32 1.0 (81, 16)
RandomSearch 8 Matern32 0.1 (201, 16)
RandomSearch 8 Matern32 0.01 (201, 16)
RandomSearch 8 Matern52 1.0 (81, 16)
RandomSearch 8 Matern52 0.1 (201, 16)
RandomSearch 8 Matern52 0.01 (201, 16)
RandomSearch 8 RBF 1.0 (41, 16)
RandomSearch 8 RBF 0.1 (161, 16)
RandomSearch 8 RBF 0.01 (161, 16)
RandomSearch 16 Matern32 1.0 (161, 16)
RandomSearch 16 Matern32 0.1 (401, 16)
RandomSearch 16 Matern32 0.01 (401, 16)
RandomSearch 16 Matern52 1.0 (161, 16)
RandomSearch 16 Matern52 0.1 (401, 16)
RandomSearch 16 Matern52 0.01 (401, 16)
RandomSearch 16 RBF 1.0 (81, 16)
RandomSearch 16 RBF 0.1 (321, 16)
RandomSearch 16 RBF 0.01 (321, 16)
RandomSear

In [5]:
for a in acquisition_functions.keys():
    for d in dimensions:
        for k in kernels:
            for l in lengthscales:
                config_and_metrics_per_seed = grouped_runs[a,d,k,l]
        
                cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                # Handling potential empty arrays
                if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
                    continue  # Skip this iteration if there's no data
                global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0])
        
                regret_per_seed = global_optimum_per_seed - best_observed_per_seed
                print(a, d, k, l, regret_per_seed.shape)
    
                mean = np.mean(regret_per_seed, axis=1)
                se = np.std(regret_per_seed, axis=1, ddof=1) / np.sqrt(regret_per_seed.shape[1])
                
                mean_plus_2se = mean + 2 * se
                mean_minus_2se = mean - 2 * se
        
                output = np.stack((cumulative_cost_per_seed.mean(axis=1), mean_minus_2se, mean, mean_plus_2se),axis=-1)
        
                np.savetxt(f"results/SE/fixed_amplitude/BayesianRegret_FixedCost_d{d}_{k}_ls{l}_{acquisition_functions[a]}.csv", output, header="cc, mean-2se, mean, mean+2se", delimiter=', ', comments='')

RandomSearch 4 Matern32 1.0 (41, 16)
RandomSearch 4 Matern32 0.1 (101, 16)
RandomSearch 4 Matern52 1.0 (41, 16)
RandomSearch 4 Matern52 0.1 (101, 16)
RandomSearch 4 RBF 1.0 (21, 16)
RandomSearch 4 RBF 0.1 (81, 16)
RandomSearch 8 Matern32 1.0 (81, 16)
RandomSearch 8 Matern32 0.1 (201, 16)
RandomSearch 8 Matern52 1.0 (81, 16)
RandomSearch 8 Matern52 0.1 (201, 16)
RandomSearch 8 RBF 1.0 (41, 16)
RandomSearch 8 RBF 0.1 (161, 16)
RandomSearch 16 Matern32 1.0 (161, 16)
RandomSearch 16 Matern32 0.1 (401, 16)
RandomSearch 16 Matern52 1.0 (161, 16)
RandomSearch 16 Matern52 0.1 (401, 16)
RandomSearch 16 RBF 1.0 (81, 16)
RandomSearch 16 RBF 0.1 (321, 16)
RandomSearch 32 Matern32 1.0 (321, 16)
RandomSearch 32 Matern32 0.1 (801, 16)
RandomSearch 32 Matern52 1.0 (321, 16)
RandomSearch 32 Matern52 0.1 (801, 16)
RandomSearch 32 RBF 1.0 (161, 16)
RandomSearch 32 RBF 0.1 (641, 16)
ExpectedImprovement 4 Matern32 1.0 (41, 16)
ExpectedImprovement 4 Matern32 0.1 (101, 16)
ExpectedImprovement 4 Matern52 1.0 